In [85]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

In [97]:
headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

page = "https://www.transfermarkt.com/wettbewerbe/fifa?ajax=yw1&page={}"



In [98]:
total_values = []
ranks_list = []
sqaud_size = []
avg_age = []
points = []
teams_list = []
confederations = []

for page_number in range(1,10):
    page_link = page.format(page_number)
    pageTree = requests.get(page_link, headers=headers)
    pageSoup = BeautifulSoup(pageTree.content, 'html.parser')
    
    ranks = pageSoup.find_all("td", {"class": "zentriert cp"})
    teams = pageSoup.find_all("td", {"class": "hauptlink"})
    squad = pageSoup.find_all("td", {"class": "zentriert"})
    t_value = pageSoup.find_all("td", {"class": "rechts"})


    for i in range(0,len(squad),5):
        sqaud_size.append(squad[i+1].text)   
        avg_age.append(squad[i+2].text)
        confederations.append(squad[i+3].text)
        points.append(squad[i+4].text) 

    for i in t_value:
        total_values.append(i.get_text())


    for i in ranks:
        ranks_list.append(i.get_text())

    for i in range(0,len(teams),2):
        teams_list.append(teams[i].text[1:])


In [121]:
rank_df = pd.DataFrame({'Rank':ranks_list,
                       'Nation':teams_list,
                       'Squad size':sqaud_size,
                       'Avg. age':avg_age,
                       'Total value':total_values,
                       'Confederation':confederations,
                       'Points':points})
rank_df

,Rank,Nation,Squad size,Avg. age,Total value,Confederation,Points
0,1,Brazil,24,28.1,€849.50m,CONMEBOL,1833
1,2,Belgium,26,26.5,€351.30m,UEFA,1827
2,3,France,23,27.3,€970.50m,UEFA,1790
3,4,Argentina,28,27.2,€585.50m,CONMEBOL,1765
4,5,England,24,26.2,€1.04bn,UEFA,1762
...,...,...,...,...,...,...,...
202,207,Guam,22,26.4,€645Th.,AFC,838
203,208,US Virgin,23,23.3,€75Th.,CONCACAF,816
204,209,B. Virgin,18,22.4,-,CONCACAF,813
205,210,Anguilla,16,26.1,-,CONCACAF,792


In [122]:
rank_df = rank_df.replace('-','00')
rank_df['Total value'] = [int(1000000000*float(x.replace('bn', ''))) if 'bn' in x else int(1000000*float(x.replace('m', ''))) if 'm' in x else int(1000*float(x.replace('Th.',''))) if 'Th.' in x else float(x) for x in rank_df['Total value'].str[1:]]
rank_df

,Rank,Nation,Squad size,Avg. age,Total value,Confederation,Points
0,1,Brazil,24,28.1,8.495000e+08,CONMEBOL,1833
1,2,Belgium,26,26.5,3.513000e+08,UEFA,1827
2,3,France,23,27.3,9.705000e+08,UEFA,1790
3,4,Argentina,28,27.2,5.855000e+08,CONMEBOL,1765
4,5,England,24,26.2,1.040000e+09,UEFA,1762
...,...,...,...,...,...,...,...
202,207,Guam,22,26.4,6.450000e+05,AFC,838
203,208,US Virgin,23,23.3,7.500000e+04,CONCACAF,816
204,209,B. Virgin,18,22.4,0.000000e+00,CONCACAF,813
205,210,Anguilla,16,26.1,0.000000e+00,CONCACAF,792


In [123]:
rank_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207 entries, 0 to 206
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Rank           207 non-null    object 
 1   Nation         207 non-null    object 
 2   Squad size     207 non-null    object 
 3   Avg. age       207 non-null    object 
 4   Total value    207 non-null    float64
 5   Confederation  207 non-null    object 
 6   Points         207 non-null    object 
dtypes: float64(1), object(6)
memory usage: 11.4+ KB


In [124]:
rank_df['Rank'] = pd.to_numeric(rank_df['Rank'].str[:-1],errors='coerce').astype(np.int64)
rank_df['Squad size'] = pd.to_numeric(rank_df['Squad size'],errors='coerce').astype(np.int64)
rank_df['Avg. age'] = pd.to_numeric(rank_df['Avg. age'],errors='coerce')
rank_df['Points'] = pd.to_numeric(rank_df['Points'],errors='coerce').astype(np.int64)
rank_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207 entries, 0 to 206
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Rank           207 non-null    int64  
 1   Nation         207 non-null    object 
 2   Squad size     207 non-null    int64  
 3   Avg. age       207 non-null    float64
 4   Total value    207 non-null    float64
 5   Confederation  207 non-null    object 
 6   Points         207 non-null    int64  
dtypes: float64(2), int64(3), object(2)
memory usage: 11.4+ KB


In [125]:
rank_df.to_csv('/Model 2/Data/rank.csv')